# Using continous / categorical / transition matrices to get recommendations / Selective Box Cox

# 06-10

- KNN Clustering 
- Using Categories from KNN to get subset from information
- Continous Features(Transition Matrices, HighLevel Features)
    - Box Cox
    - Centering and Scaling
    - Remove Constant Features
    - Euclidean Distance
- Categorical Features
    - Hot Encoding
    - Jaccard

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf 

from ipynb.fs.full.SearchInfo import searchByID, searchBySpotify, getTrackSpotifyInfo
from ipynb.fs.full.Utils import append_df_to_csv,transformToPercentage

In [2]:
transitionMatrices = pd.read_csv('./data/unified/07-transitionsClustered.csv')
transitionMatrices = transitionMatrices[['id','category']]

In [3]:
cover = loadCoverGroups()
listCovers = cover[0]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [4]:
cover

[[180849,
  6001027,
  1686718,
  5000013,
  5000001,
  5000007,
  5000008,
  5000023,
  5000017,
  5000019,
  5000020,
  5000018,
  5000011,
  5000012],
 [6001971, 6001984],
 [6002492, 6002491],
 [6000699, 6001165, 6000182],
 [6000104, 6000273],
 [6000768, 6000784],
 [6001931, 6001748],
 [6002043, 6002073],
 [6002244, 6000194],
 [6001209, 6000606],
 [6000352, 6001248],
 [6000549, 6001508, 6001242],
 [6001918, 6001997],
 [6001651, 6001900],
 [6000263, 6001570, 6002374],
 [6001822, 6001653],
 [6000763, 6000668],
 [6000764, 6000667],
 [6001543, 6001541],
 [6000762, 6000773],
 [6000771, 6000670],
 [6001747, 6001952],
 [6000548, 6002203],
 [6000172, 6002133],
 [6001762, 6001675],
 [6002147, 6002146],
 [6000767, 6000775],
 [6000766, 6000673],
 [6001506, 6000529],
 [6000397, 6000409],
 [6001914, 6002001],
 [6002482, 6002454],
 [6000765, 6000783],
 [6000761, 6000782],
 [6000814, 6000813],
 [6001224, 6000140]]

In [5]:
originalSongId

180849

In [6]:
df = pd.read_csv("./data/unified/05-allDataContinousCategoricalTransition.csv")
df = df.drop(columns=['Unnamed: 0', 'index'])

In [7]:
transitionMatrices

,id,category
0,23,1
1,105,0
2,118,2
3,120,1
4,292,2
...,...,...
5975,6002496,0
5976,6002497,1
5977,6002498,1
5978,6002499,1


In [8]:
df = pd.merge(df,transitionMatrices, on='id')

In [9]:
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]

In [10]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2,category
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
12,635,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,6002477,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0
5967,6002491,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [11]:
df = df.drop(columns=['category'])

In [12]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
12,635,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,6002477,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5967,6002491,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,588,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
12,635,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,6002477,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5967,6002491,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [15]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.726,0.537,0.1140,0.316000,0.0707,0.882,96.889,0.101351,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.550,0.145,0.0332,0.913000,0.1180,0.138,103.918,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,0.700,0.710,0.0532,0.327000,0.1290,0.582,167.971,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2166,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2167,0.665,0.515,0.0442,0.454000,0.3670,0.805,93.082,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2168,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Pipeline

In [16]:
dfEuclidean, dfJaccard = getEuclideanJaccardDf(df)

In [17]:
dfEuclidean

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,997,998,999,acousticness,danceability,energy,liveness,speechiness,tempo,valence
0,0.185185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000458,0.582,0.744,0.0405,0.0336,133.344,0.674
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000013,0.577,0.890,0.0954,0.0345,132.397,0.643
2,0.007634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.038000,0.664,0.787,0.1930,0.0239,102.952,0.585
3,0.101351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.316000,0.726,0.537,0.0707,0.1140,96.889,0.882
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.913000,0.550,0.145,0.1180,0.0332,103.918,0.138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.327000,0.700,0.710,0.1290,0.0532,167.971,0.582
2166,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.390000,0.760,0.838,0.1660,0.0510,159.938,0.863
2167,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.454000,0.665,0.515,0.3670,0.0442,93.082,0.805
2168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.113000,0.555,0.578,0.0747,0.0277,83.382,0.333


In [18]:
dfJaccard

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2166,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2167,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# dfEuclidean = applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
dfEuclidean = dfEuclidean.copy()
dfEuclidean = removeConstantFeatures(dfEuclidean)
# dfEuclidean= applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
dfEuclidean= applyBoxCoxAllFeatures(dfEuclidean)
dfEuclidean = centerAndScale(dfEuclidean)

mean:  [ 0.  0.  0. -0.  0. -0. -0. -0. -0.  0. -0.  0.  0. -0.  0.  0. -0. -0.
 -0.  0. -0.  0. -0. -0.  0. -0. -0.  0.  0. -0. -0.  0.  0.  0.  0.  0.
  0.  0. -0. -0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0. -0. -0. -0. -0.
 -0.  0. -0. -0.  0.  0.  0.  0.  0. -0. -0.  0. -0.  0.  0. -0. -0.  0.
 -0.  0. -0. -0.  0.  0. -0. -0. -0.  0. -0. -0.  0.  0.  0. -0. -0. -0.
 -0.  0.  0. -0. -0.  0. -0. -0. -0. -0.  0. -0.  0.  0. -0. -0.  0. -0.
  0. -0. -0. -0.  0.  0. -0. -0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.
  0. -0. -0. -0. -0.  0.  0.  0. -0. -0. -0.  0.  0.  0. -0.  0.  0.  0.
 -0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0.  0. -0.  0.  0.  0.  0. -0. -0.  0. -0.  0. -0. -0. -0.
  0. -0.  0.  0.  0.  0. -0. -0.  0. -0.  0. -0. -0. -0.  0. -0.  0. -0.
  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0.
 -0. -0.  0. -0.  0.  0. -0. -0.  0.  0. -0.  0.  0.  0.  0. -0.  0. -0.
  0.  0. -0.  0.  0. -0.  0.  0. -0.  0.  0.

### Original Song & Cover Song

In [20]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfEuclidean)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfEuclidean)

### Get Rankings Euclidean

In [21]:
ranksDFEuclidean  = applyEuclideanDistance(dfEuclidean, ids, originalSongVector)
ranksDFEuclidean

distance


,id,rank
0,180849,0.000000
1,748267,23.963141
2,88594,24.281379
3,610562,24.320615
4,6001027,24.322165
...,...,...
2165,6002206,132.767245
2166,6000799,153.322412
2167,79223,174.184124
2168,6000784,186.593048


In [22]:
ranksDFEuclidean[ranksDFEuclidean.id == originalSongId]

,id,rank
0,180849,0.0


In [23]:
ranksDFEuclidean[ranksDFEuclidean['id']==coverSongId]

,id,rank
4,6001027,24.322165


In [24]:
ranksDFEuclidean.loc[ranksDFEuclidean.id.isin(listCovers)]

,id,rank
0,180849,0.000000
4,6001027,24.322165
16,5000019,24.576847
26,5000017,24.635310
136,5000013,24.931357
424,1686718,25.192736
431,5000012,25.199934
756,5000011,25.541282
1182,5000001,26.534447
1323,5000018,27.331125


### Get Rankings Jaccard

In [25]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfJaccard)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfJaccard)

In [26]:
ranksDFJaccard  = applyJaccard(dfJaccard, ids, originalSongVector)

In [27]:
ranksDFJaccard

,id,rank
0,180849,0.0
1,6001027,0.0
2,450492,0.0
3,466649,0.0
4,1029726,0.0
...,...,...
2165,780945,1.0
2166,388558,1.0
2167,821402,1.0
2168,6001417,1.0


In [28]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
1,6001027,0.0


In [29]:
ranksDFJaccard.loc[ranksDFJaccard.id.isin(listCovers)]

,id,rank
0,180849,0.000000
1,6001027,0.000000
36,5000001,0.333333
37,5000017,0.333333
38,5000019,0.333333
40,5000020,0.333333
327,5000018,0.571429
330,5000013,0.571429
335,5000008,0.571429
640,1686718,0.571429


### Experiment to unify ranks

In [30]:
ranks = pd.merge(ranksDFJaccard,ranksDFEuclidean, on='id')

In [31]:
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']

In [32]:
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)

In [33]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
1,6001027,0.000000,24.322165,24.322165
2,748267,0.571429,23.963141,24.534570
3,2065851,0.333333,24.325463,24.658796
4,5000019,0.333333,24.576847,24.910181
5,732460,0.571429,24.369539,24.940967
6,5000017,0.333333,24.635310,24.968643
7,2689908,0.571429,24.421339,24.992768
8,6000141,0.571429,24.441914,25.013343
9,88594,0.750000,24.281379,25.031379


In [34]:
ranks.loc[ranks.id.isin(listCovers)]

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
1,6001027,0.000000,24.322165,24.322165
4,5000019,0.333333,24.576847,24.910181
6,5000017,0.333333,24.635310,24.968643
112,5000013,0.571429,24.931357,25.502785
301,1686718,0.571429,25.192736,25.764165
489,5000012,0.750000,25.199934,25.949934
787,5000011,0.750000,25.541282,26.291282
1102,5000001,0.333333,26.534447,26.867780
1298,5000018,0.571429,27.331125,27.902553


In [35]:
ranks[ranks['id']==coverSongId]

,id,rank_x,rank_y,new_rank
1,6001027,0.0,24.322165,24.322165


In [36]:
ranks.rename(columns = {'new_rank':'rank'}, inplace = True)

In [38]:
# To extract covers info change path in util if needed
# ranks = transformToPercentage(ranks.loc[ranks.id.isin(listCovers)], '0700')
# ranks

,id_chord,id_spotify_track,name,release_date,percentage,rank,position,id_experiment
0,180849,3jButwtJMLx3Ub61BfRiHh,the scientist,2022-06-03,100.000,0.000000,0,0700
1,6001027,75JFxkI2RXiU7L9VXzMkle,NaN,NaN,75.678,24.322165,1,0700
2,5000019,49ilSrZNQcle6PYw8rtvzM,NaN,NaN,75.090,24.910181,4,0700
3,5000017,34dnNAUoIPcwnK0RtVMBWZ,NaN,NaN,75.031,24.968643,6,0700
4,5000013,2LbHYfTYW6TipHInIjqdaQ,NaN,NaN,74.497,25.502785,112,0700
5,1686718,2LbHYfTYW6TipHInIjqdaQ,the scientist,2015-04-28,74.236,25.764165,301,0700
6,5000012,7tn1tcxenMwDXrYTOjS3z1,NaN,NaN,74.050,25.949934,489,0700
7,5000011,5FoM2ZWwhgTxHFFh2stUw6,NaN,NaN,73.709,26.291282,787,0700
8,5000001,025vpSSyndhVZezl8rUF6L,NaN,NaN,73.132,26.867780,1102,0700
9,5000018,49Nzxf1LskunwmZ6EyWUew,NaN,NaN,72.097,27.902553,1298,0700
